In [1]:
from iism_lab2 import a5_generator
import numpy as np


def get_a5_generator():
    S19 = np.random.randint(2, size=19)
    S22 = np.random.randint(2, size=22)
    S23 = np.random.randint(2, size=23)
    gen = a5_generator(S19, S22, S23)
    return gen


def uniform_psi(gen, n=10):
    while True:
        yield (int(''.join([str(next(gen)) for _ in range(n)]), 2) + 1) / (2 ** n + 1)

$$\int_{88}^{99} \ln{(x)}\cdot\sin{(x)} dx = \int_{88}^{99} \frac{1}{11} \cdot 11 \cdot \ln{(x)}\cdot \sin{(x)} = E\{11\cdot\ln{(x)}\cdot\sin{(x)}\}\approx \frac{1}{n}\sum_{i=1}^{n} 11 \cdot \ln{(x_i)}\sin{(x_i)}$$

In [15]:
psi = uniform_psi(get_a5_generator())
exps = np.array([11 * next(psi) + 88 for _ in range(2000)])
print(np.mean(exps))

93.521962439


$4.2818$

In [16]:
value = np.mean(11 * np.log(exps) * np.sin(exps))
print(value)

4.3075411555


In [29]:
exps = np.random.uniform(low=88, high=99, size=100000)
value = np.mean(11 * np.log(exps) * np.sin(exps))
print(value)

4.26459149058


Теперь решим $$\iint\limits_{|x| + |y| < 1}\frac{x^3+3xy}{e^{-y}}dxdy = \iint\limits_{|x| + |y| < 1} \frac{1}{2} \cdot 2 \cdot \frac{x^3 + 3xy}{e^{-y}}dxdy \approx \frac{1}{n} \sum\limits_{i=1}^{n} 2 \cdot \frac{x_i^3 + 3x_iy_i}{e^{-y_i}}$$

Правильное значение: $0$

In [4]:
exps = np.random.uniform(low=-1/2**0.5, high=1/2**0.5, size=(10000, 2))
M = np.array([[np.cos(np.pi / 4), -np.sin(np.pi / 4)], 
              [np.sin(np.pi / 4), np.cos(np.pi / 4)]])
for i in range(len(exps)):
    exps[i] = (M.dot(exps[i].reshape((2, 1)))).reshape((1, 2))
x, y = exps.reshape((2, 10000))
value = np.mean(2 * (x ** 3 + 3 * x * y) / np.exp(-y))
print(value)

1.20185757916e-05


Решим систему $x = Ax + f$ методом Монте-Карло

In [2]:
data = np.genfromtxt('data/Юнусов.txt', delimiter=' ')
A, f = data[:-1], data[-1]
print(A.shape, f.shape)

(37, 37) (37,)


Как видим, все собственные значения матрицы меньше 1

In [3]:
np.max(A)

0.0576739

In [11]:
from copy import deepcopy
x_new = deepcopy(f)
x_prev = deepcopy(f) - 1000
while abs(np.max(x_prev - x_new)) > 0.00001:
    x_prev = deepcopy(x_new)
    x_new = A.dot(x_new) + f
    print(np.max(x_new - A.dot(x_new) - f))

-0.00875601503225
-0.0116750194937
-0.00238197167137
-0.000603520518509
-0.000163411487571
-4.45742467821e-05
-1.21214950313e-05
-3.27321585059e-06
-8.8382187885e-07


In [32]:
N, m = 1000, 10000
H = np.eye(37)

result = []
for l in range(37):
    Q, ksi = [None] * (N + 1), [0.] * m
    for j in range(m):
        i = []
        for k in range(N + 1):
            # Предполагаем, что начальные состояния и 
            # переходы равновероятны
            alpha = np.random.randint(low=0, high=36)
            i.append(alpha)
        Q[0] = H[l][i[0]] * 37
        for k in range(1, N + 1):
            Q[k] = Q[k - 1] * A[i[k - 1]][i[k]] * 37
        for k in range(N + 1):
            ksi[j] += Q[k] * f[i[k]]
    result.append(np.mean(ksi))

result = np.array(result)
print(result)
print(np.max(result - A.dot(result) - f))

[ 10.85296693   0.84128871   9.30525122   7.33215852   3.3569751
   2.63917693   3.85993334   8.38740367  10.24436007   6.67062109
   5.86132497   7.1921812    1.73229571   7.16899014   0.94713827
   0.55360973   0.10811483   1.24843684   6.30589079   7.33458821
  10.91506338   7.0427061    3.80036138   2.33197014   8.26718437
   2.47309046   8.59810746   4.52422117   8.22380042   7.36867354
   7.00929838   5.30436508   7.24559433   0.81625808   0.2610028
   2.29364764   0.        ]
1.13205175471


In [33]:
print(result - A.dot(result) - f)

[ 1.02795646  0.02165248  0.34585369  0.49356945  0.39715734 -0.10640843
 -0.09654002  0.29834721  1.05423693  0.33653769  0.32726278  0.67008707
  0.1158134   0.34931456  0.21926865 -0.0534122  -0.0170049   0.43311519
 -0.21014583  0.84284058  1.13205175 -0.65714586  0.06795861  0.01351712
 -0.14045664  0.20113266  0.90681266 -0.02401769 -0.597361    0.32508542
  0.5363362   0.13705238  0.17533025  0.63487407 -0.41877157 -0.40547782
 -2.85439163]
